In [1]:
import numpy as np
import h5py 
import matplotlib.pyplot as plt
import time
from IPython import display

In [ ]:
# function to load two of the 10 classes (c1 is for Y=-1 and c2 is for Y=+1)
def loaddigitdata(c1,c2,m):
#     f = h5py.File('/usr/local/cs171/uspsall73.mat','r') 
    data = f.get('data') 
    data = np.array(data).astype(float)
    X = np.concatenate((data[c1,:,:],data[c2,:,:]))
    Y = np.concatenate((-np.ones((data.shape[1])),np.ones((data.shape[1]))))
    
    rs = np.random.RandomState(seed=132857) # setting seed so that dataset is consistent
    p = rs.permutation(X.shape[0])
    X = X[p] # this and next line make copies, but that's okay given how small our dataset is
    Y = Y[p]
    
    trainX = X[0:m,:] # use the first m (after shuffling) for training
    trainY = Y[0:m,np.newaxis]
    validX = X[m:,:] # use the rest for validation
    validY = Y[m:,np.newaxis]
    return (trainX,trainY,validX,validY)


def drawexample(x,ax=None): # takes an x *vector* and draws the image it encodes
    if ax is None:
        plt.imshow(np.reshape(x,(16,16)).T,cmap='gist_yarg')
    else:
        ax.imshow(np.reshape(x,(16,16)).T,cmap='gist_yarg')

In [ ]:
# load the data, to differentiate between 7s and 9s
# we will use on 1100 examples for training (50% of the data) and the other half for hold-out validation
(trainX,trainY,validX,validY) = loaddigitdata(6,8,1100)
means = trainX.mean(axis=0)
stddevs = trainX.std(axis=0)
stddevs[stddevs<1e-6] = 1.0
trainX = (trainX-means)/stddevs # z-score normalization
validX = (validX-means)/stddevs # apply same transformation to validation set

In [ ]:
# Convert this cell to a code cell if you wish to see each of the examples, plotted
f = plt.figure()
f.set_size_inches(8,8)

ax = f.add_subplot(111)
plt.ion()
f.canvas.draw()
for exi in range(trainX.shape[0]):
    display.clear_output(wait=True)
    drawexample(trainX[exi,:])
    digitid = (9 if trainY[exi]>0 else 7)
    ax.set_title('y = '+str(int(trainY[exi]))+" ["+str(digitid)+"]")
    display.display(f)
    #time.sleep(0.1)

In [ ]:

def backwardProp(X, Y,  Wts, fx, lam):
#     w1 = np.array([[0.80, 0.10, 0.30], [0.10, 0.00, 0.30], [0.00 ,-1.00, 0.50]])
#     W2 = np.array([[-1.00, 0.50, -0.40, 1.10], [1.00, -0.2, -1.5, 0.20]])
#     W3 = np.array([[-1, -3, 2]])
    
#     X = np.array([[2, -1], [1,3], [0, -2]])

#     fx = nneval(X,Wts)

   
    DLF = -1 * (1 - (1/(1 + np.exp(-Y*fx))))*Y

    W1 = Wts[0]
    W2 = Wts[1]
    W3 = Wts[2]

    #####THINK ON THIS
    delta3 = np.ones((len(X),1))

    d2 = delta3.dot(W3)


    delta2 = d2[:, 1:] * (1 - np.tanh(Z2)**2)


    d1 = delta2.dot(W2)


    delta1 = d1[:, 1:] * (1 - np.tanh(Z1)**2)


    #Batch gradients
    dw1 = np.zeros((len(W1), len(W1[0])))
    dw2 = np.zeros((len(W2), len(W2[0])))
    dw3 = np.zeros((len(W3), len(W3[0])))
 
    for i in range(len(X)):
        dw3 += DLF[i]*np.matmul(np.atleast_2d(delta3[i]).T,np.atleast_2d(A2[i]))
        
        
        dw2 += DLF[i]*np.matmul(np.atleast_2d(delta2[i]).T,np.atleast_2d(A1[i]))

        
        dw1 += DLF[i]*np.matmul(np.atleast_2d(delta1[i]).T,np.atleast_2d(A0[i])) 
    
    
    dw3 += 2*lam*W3
    dw2 += 2*lam*W2
    dw1 += 2*lam*W1
    
    return [dw1,dw2,dw3] 








def nneval(X,Wts):
    # X is m-by-n (no "extra 1s" features added)
    # Wts is a python array of 3 matrices
    # Wts[0] is (n+1)-by-h (if there are h hidden units in the first layer)
    # Wts[1] is (h+1)-by-q (if there are q hidden units in the second layer)
    # Wts[2] is (q+1)-by-1
    # This function  returns  matrix that is (m,1) -- the discriminant value for each example/row in X
    
    global A0, A1, A2
    global Z1, Z2
    
    inputs = len(X[0])
    
    layer1HiddenUnitsCnt = len(Wts[0])
    
    layer2HiiddenUnitsCnt = len(Wts[1])
    
    
    A0 = np.empty((0, inputs + 1))
    
    A1 = np.empty((0, layer1HiddenUnitsCnt + 1))
    
    A2 = np.empty((0, layer2HiiddenUnitsCnt + 1))
    
    
    Z1 = np.empty((0, layer2HiiddenUnitsCnt))
    
    Z2 = np.empty((0, layer2HiiddenUnitsCnt))
    
    
    X  = np.concatenate((np.ones((X.shape[0], 1)),X), axis = 1)
    A0 = X
    
    
    Z1 =  np.dot(A0, Wts[0].transpose())
    A1 =  np.tanh(Z1)
    A1  = np.concatenate((np.ones((A1.shape[0], 1)),A1), axis = 1)
    
    
    Z2 =  np.dot(A1, Wts[1].transpose())
    A2 =  np.tanh(Z2)
    A2  = np.concatenate((np.ones((A2.shape[0], 1)),A2), axis = 1)
    
    Z3 =  np.dot(A2, Wts[2].transpose())
    

    
    
    return Z3
def trainneuralnet(X,Y,nhid,lam,printinfo=False):
    
    threshHold = 0.000001
    # The number of examples (m) and number of input dimensions (n) of the input data
    (m,n) = X.shape
    
    # This is the code that initializes the weight matrics:
    W1 = (np.random.rand(nhid,n+1)*2-1)*np.sqrt(6.0/(n+nhid+1)) # weights to each hidden unit from the inputs (plus the added offset unit)
    W2 = (np.random.rand(nhid//2,nhid+1)*2-1)*np.sqrt(6.0/(nhid+nhid//2+1))
    W3 = (np.random.rand(1,nhid//2+1)*2-1)*np.sqrt(6.0/(nhid//2+2)) # weights to the single output unit from the hidden units (plus the offset unit)
    W1[:,0] = 0 # good initializations for the constant terms
    W2[:,0] = 0 #-nhid/2.0
    W3[:,0] = 0 #-nhid/4.0
    
    Wts = [W1,W2,W3] 
    
    Eg2=1 # a variable that is kept around for updating eta
    
    
    ##1- Forward Propagation Results
    fx = nneval(X,Wts)
    
 
    
    
    
    ##2-- Calculate Total Loss
    sig = 1/(1 + np.exp(-Y*fx))
    
    L1 = np.sum(-np.log(sig))
    
    ##2-a Regularization Part in the Loss
    L2 = lam*(np.sum(Wts[0]*Wts[0]) + np.sum(Wts[1]*Wts[1]) + np.sum(Wts[2]*Wts[2]))
    
    
    PreviousLoss = L1 + L2
    
    # print(PreviousLoss)
    
    # updates = backwardProp(X, Y,  Wts, fx, lam)
    
    
    
    
    i = 0
    
    while i <= 1700:
        
       
     
        
        updates = backwardProp(X, Y,  Wts, fx, lam)
        
        sumofgrad2 = (np.sum(updates[0]*updates[0]) + np.sum(updates[1]*updates[1]) + np.sum(updates[2]*updates[2]))
        
        Eg2 = 0.9*Eg2 + 0.1*sumofgrad2
        eta = 0.01/(np.sqrt((1e-10+Eg2)))
        
        
        for j in range(3):
            Wts[j] = Wts[j] - (eta * updates[j])
            
    
        
        fx_new = nneval(X,Wts)
        
        ##2-- Calculate Total Loss
        sig = 1/(1 + np.exp(-Y*fx_new))
    
        L1 = np.sum(-np.log(sig))
    
        ##2-a Regularization Part in the Loss
        L2 = lam*(np.sum(Wts[0]*Wts[0]) + np.sum(Wts[1]*Wts[1]) + np.sum(Wts[2]*Wts[2]))
    
        CurrentLoss = L1 + L2
        
        # print(PreviousLoss)
        
        ##Check the deffence b/w the current and previous Loss After 10 iteratins
        if i > 0 and i % 10 == 0 and abs(CurrentLoss - PreviousLoss) < threshHold:
            break
        
        PreviousLoss = CurrentLoss
        
        
        
        fx = fx_new
        i +=1
        

    return Wts 

In [ ]:
%%time
Wts = trainneuralnet(trainX,trainY,16,1e-5,True)

In [ ]:
## Code added to do logistic regression, just for a comparison
def learnlogreg_sklearn(X,Y,regstr=0.0,penalizeb=False):
    X = np.hstack((np.ones((X.shape[0],1)),X))
    Y = Y[:,0]
    from sklearn.linear_model import LogisticRegression as lr
    if regstr==0.0:
        pen = 'none'
        C = np.inf
    else:
        pen = 'l2'
        C = 2.0/regstr

    Y = (Y>0.5).astype(int)
    if penalizeb:
        lrres = lr(tol=1e-6,max_iter=100000,penalty=pen,C=C,
                   fit_intercept=False,
                   solver='newton-cg',multi_class='multinomial').fit(X,Y)
        w = lrres.coef_[0,:]
    else:
        lrres = lr(tol=1e-6,max_iter=100000,penalty=pen,C=C,
                   fit_intercept=True,
                   solver='newton-cg',multi_class='multinomial').fit(X[:,1:],Y)
        w = np.hstack((lrres.intercept_,lrres.coef_[0,:]))
    return w

def predictlogreg(X,w):
    return np.hstack((np.ones((X.shape[0],1)),X))@w

In [ ]:
# Code to update the plot as new results are generated
def setupfig():
    f = plt.figure()
    f.set_size_inches(8,8)
    ax = f.add_subplot(111)
    plt.ion()
    f.canvas.draw()
    return (f,ax)

def plotit(lams,nhiddens,erates,f,ax):
    ax.clear()
    for i in range(nhiddens.shape[0]):
        ax.plot(lams,erates[:,i],'*-')
    ax.set_yscale('log',subs=[1,2,3,4,5,6,7,8,9])
    ax.set_yticks([0.1,0.01])
    ax.set_xscale('log')
    f.canvas.draw()
    ax.set_xlabel('lambda')
    ax.set_ylabel('validation error rate')
    ax.legend([(('# hidden units = '+str(x)+','+str(x//2)) if x>0 else 'logistic regression') for x in nhiddens])
    display.display(f)
    display.clear_output(wait=True)
    

In [ ]:
%%time
def errorrate(Y,predy):
    predy = np.sign(predy)
    return (predy!=Y).mean()
    
def multirestart(trainX,trainY,nhid,lam,ntries):
    besterrsc = 2.0
    for i in range(ntries):
        Wts = trainneuralnet(trainX,trainY,nhid,lam)
        errsc = errorrate(trainY,nneval(trainX,Wts))
        if errsc<besterrsc:
            returnWts = Wts
            besterrsc = errsc
    return returnWts

nhiddens = np.array([0,2,8,16])
lams = np.logspace(-12,3,10)
erates = np.empty([lams.shape[0],nhiddens.shape[0]])
erates[:,:] = np.nan

(f,ax) = setupfig()
    
for ni, nhid in enumerate(nhiddens):
    for li, lam in reversed(list(enumerate(lams))):
        if nhid==0:
            w = learnlogreg_sklearn(trainX,trainY,lam)
            predy = predictlogreg(validX,w)[:,np.newaxis]
        else:
            Wts = multirestart(trainX,trainY,nhid,lam,2) #trainneuralnet(trainX,trainY,nhid,lam)
            predy = nneval(validX,Wts)
        erates[li,ni] = errorrate(validY,predy)
        
        plotit(lams,nhiddens,erates,f,ax)